In [1]:
# Libraries
import tensorflow
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, ConcatDataset
from transformers import AutoTokenizer, BertTokenizer 

import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sn
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

2023-05-30 08:04:18.778243: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# Function untuk konversi waktu dari detik ke jam, menit, detik
def evl_time(t):
    min, sec= divmod(t, 60)
    hr, min = divmod(min, 60)
    return int(hr), int(min), int(sec)

# Function untuk menyimpan model
def save_model(model, name):
    torch.save(model.state_dict(), name)

# Function untuk load model
def load_model(model, path):
    model.load_state_dict(torch.load(path), strict=False)
    return model

In [3]:
# Mengubah dataset menjadi beberapa mini-batch
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors = pad_sequence(ids_tensors, batch_first=True)

    segments_tensors = [s[2] for s in samples]
    segments_tensors = pad_sequence(segments_tensors, batch_first=True)

    label_ids = torch.stack([s[3] for s in samples])
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1)

    return ids_tensors, segments_tensors, masks_tensors, label_ids

In [4]:
# Function untuk pelatihan model
def train_model_ABSA(loader, epochs, model_name):
    history = {'loss' : []}
    all_data = len(loader)
    
    # Pengulangan epoch
    for epoch in range(epochs):
        finish_data = 0
        losses = []
        current_times = []
        
        # Pengulangan setiap mini-batch
        for data in loader:
            t0 = time.time()
            ids_tensors, segments_tensors, masks_tensors, label_ids = data
            ids_tensors = ids_tensors.to(DEVICE)
            segments_tensors = segments_tensors.to(DEVICE)
            label_ids = label_ids.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            loss = model_ABSA(ids_tensors=ids_tensors, lable_tensors=label_ids, masks_tensors=masks_tensors, segments_tensors=segments_tensors)
            losses.append(loss.item())
            loss.backward()
            optimizer_ABSA.step()
            optimizer_ABSA.zero_grad()

            finish_data += 1
            current_times.append(round(time.time()-t0,3))
            current = np.mean(current_times)
            hr, min, sec = evl_time(current*(all_data-finish_data) + current*all_data*(epochs-epoch-1))
            print('epoch:', epoch+1, "/" , epochs," batch:", finish_data, "/" , all_data, " loss:", np.mean(losses), " hr:", hr, " min:", min," sec:", sec)

        history['loss'].append(np.mean(losses))
        save_model(model_ABSA, model_name)
    
    return history

# Function untuk pengujian model
def test_model_ABSA(loader):
    pred = []
    truth = []
    
    with torch.no_grad():
        # Pengulangan setiap mini-batch
        for data in loader:
            ids_tensors, segments_tensors, masks_tensors, label_ids = data
            ids_tensors = ids_tensors.to(DEVICE)
            segments_tensors = segments_tensors.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            outputs = model_ABSA(ids_tensors, None, masks_tensors=masks_tensors, segments_tensors=segments_tensors)
            
            _, predictions = torch.max(outputs, dim=1)

            pred += list([int(i) for i in predictions])
            truth += list([int(i) for i in label_ids])

    return truth, pred

In [5]:
# Function untuk menampilkan confusion matrix
def draw_c_matrix(c_matrix, title="test"):
    aix = []
    
    for y in range(len(c_matrix)):
        aix.append(y)
    df_cm = pd.DataFrame(c_matrix, aix, aix)
    sn.heatmap(df_cm, annot=True, fmt='g', cmap='Blues')
    plt.ylabel("prediction")
    plt.xlabel("ground truth")
    plt.title(title)
    plt.show()

In [6]:
# Menentukan device yang akan digunakan untuk melakukan komputasi
DEVICE = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

print("Tensorflow\t: ", tensorflow.__version__)
print("Torch\t\t: ", torch.__version__)
print("Device\t\t: ", DEVICE)
print("GPU\t\t: ", torch.cuda.get_device_name())
print("CUDA\t\t: ", torch.version.cuda)

Tensorflow	:  2.4.1
Torch		:  1.4.0
Device		:  cuda:3
GPU		:  Tesla T4
CUDA		:  10.1


In [7]:
# Inisialisasi hyperparameter
bs = 8
lr = 0.00002
epoch = 8

In [8]:
# Inisialisasi pre-trained model BERT
from ABSA_SentimentMultiEmiten.model.bert import bert_ABSA
from ABSA_SentimentMultiEmiten.data.dataset import dataset_ABSA

model_name = "bert-multilingual.pkl"
pretrained_model_name = "bert-base-multilingual-uncased"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
model_ABSA = bert_ABSA(pretrained_model_name)
model_ABSA.to(DEVICE)
optimizer_ABSA = torch.optim.Adam(model_ABSA.parameters(), lr=lr)

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/672M [00:00<?, ?B/s]

In [9]:
# Inisialisasi dataset
emiten_train_ds = dataset_ABSA(pd.read_csv("data_experiment/data_balance_experiment_training.csv"), tokenizer)
emiten_test_ds = dataset_ABSA(pd.read_csv("data_experiment/data_balance_experiment_testing.csv"), tokenizer)

train_ds = ConcatDataset([emiten_train_ds])
test_ds = ConcatDataset([emiten_test_ds])

# Mengubah dataset menjadi mini-batch
train_loader = DataLoader(train_ds, batch_size=bs, collate_fn=create_mini_batch, shuffle = True)
test_loader = DataLoader(test_ds, batch_size=bs, collate_fn=create_mini_batch, shuffle = True)

In [10]:
# Proses training model
history = %time train_model_ABSA(train_loader, epoch, model_name)

epoch: 1 / 8  batch: 1 / 1224  loss: 1.0323455333709717  hr: 0  min: 54  sec: 59
epoch: 1 / 8  batch: 2 / 1224  loss: 1.0240721106529236  hr: 0  min: 44  sec: 3
epoch: 1 / 8  batch: 3 / 1224  loss: 1.0493390560150146  hr: 0  min: 37  sec: 54
epoch: 1 / 8  batch: 4 / 1224  loss: 1.0748914778232574  hr: 0  min: 40  sec: 37
epoch: 1 / 8  batch: 5 / 1224  loss: 1.05612313747406  hr: 0  min: 42  sec: 42
epoch: 1 / 8  batch: 6 / 1224  loss: 1.0894128481547039  hr: 0  min: 44  sec: 36
epoch: 1 / 8  batch: 7 / 1224  loss: 1.0925324644361223  hr: 0  min: 47  sec: 34
epoch: 1 / 8  batch: 8 / 1224  loss: 1.1127460449934006  hr: 0  min: 44  sec: 46
epoch: 1 / 8  batch: 9 / 1224  loss: 1.1149984465705023  hr: 0  min: 48  sec: 27
epoch: 1 / 8  batch: 10 / 1224  loss: 1.0897318482398988  hr: 0  min: 49  sec: 57
epoch: 1 / 8  batch: 11 / 1224  loss: 1.0989424857226284  hr: 0  min: 47  sec: 17
epoch: 1 / 8  batch: 12 / 1224  loss: 1.0947034358978271  hr: 0  min: 47  sec: 55
epoch: 1 / 8  batch: 13 / 12

epoch: 1 / 8  batch: 103 / 1224  loss: 0.9487540820269909  hr: 0  min: 42  sec: 21
epoch: 1 / 8  batch: 104 / 1224  loss: 0.947349200455042  hr: 0  min: 42  sec: 27
epoch: 1 / 8  batch: 105 / 1224  loss: 0.9470345729873294  hr: 0  min: 42  sec: 28
epoch: 1 / 8  batch: 106 / 1224  loss: 0.9447285875959216  hr: 0  min: 42  sec: 32
epoch: 1 / 8  batch: 107 / 1224  loss: 0.9411595725567541  hr: 0  min: 42  sec: 19
epoch: 1 / 8  batch: 108 / 1224  loss: 0.9419873345781256  hr: 0  min: 42  sec: 9
epoch: 1 / 8  batch: 109 / 1224  loss: 0.9379602266014169  hr: 0  min: 42  sec: 1
epoch: 1 / 8  batch: 110 / 1224  loss: 0.9361071825027466  hr: 0  min: 41  sec: 49
epoch: 1 / 8  batch: 111 / 1224  loss: 0.9370604689056808  hr: 0  min: 41  sec: 58
epoch: 1 / 8  batch: 112 / 1224  loss: 0.9353495266820703  hr: 0  min: 41  sec: 47
epoch: 1 / 8  batch: 113 / 1224  loss: 0.9365522655765567  hr: 0  min: 42  sec: 1
epoch: 1 / 8  batch: 114 / 1224  loss: 0.9342839524411318  hr: 0  min: 41  sec: 49
epoch: 1

epoch: 1 / 8  batch: 203 / 1224  loss: 0.8637804883747853  hr: 0  min: 41  sec: 13
epoch: 1 / 8  batch: 204 / 1224  loss: 0.8635427792574845  hr: 0  min: 41  sec: 7
epoch: 1 / 8  batch: 205 / 1224  loss: 0.8625908312274189  hr: 0  min: 41  sec: 0
epoch: 1 / 8  batch: 206 / 1224  loss: 0.8614870178178676  hr: 0  min: 41  sec: 3
epoch: 1 / 8  batch: 207 / 1224  loss: 0.8615674608283572  hr: 0  min: 40  sec: 56
epoch: 1 / 8  batch: 208 / 1224  loss: 0.8631931868596718  hr: 0  min: 40  sec: 59
epoch: 1 / 8  batch: 209 / 1224  loss: 0.8620808314193379  hr: 0  min: 40  sec: 53
epoch: 1 / 8  batch: 210 / 1224  loss: 0.8617771564494996  hr: 0  min: 40  sec: 46
epoch: 1 / 8  batch: 211 / 1224  loss: 0.8610483674923955  hr: 0  min: 40  sec: 53
epoch: 1 / 8  batch: 212 / 1224  loss: 0.8600958224456265  hr: 0  min: 40  sec: 56
epoch: 1 / 8  batch: 213 / 1224  loss: 0.8604805703174341  hr: 0  min: 40  sec: 51
epoch: 1 / 8  batch: 214 / 1224  loss: 0.8611291414387873  hr: 0  min: 40  sec: 46
epoch: 

epoch: 1 / 8  batch: 302 / 1224  loss: 0.8104353335914233  hr: 0  min: 40  sec: 50
epoch: 1 / 8  batch: 303 / 1224  loss: 0.8093919678293046  hr: 0  min: 40  sec: 52
epoch: 1 / 8  batch: 304 / 1224  loss: 0.8085357800714279  hr: 0  min: 40  sec: 53
epoch: 1 / 8  batch: 305 / 1224  loss: 0.8083885784031914  hr: 0  min: 40  sec: 49
epoch: 1 / 8  batch: 306 / 1224  loss: 0.8071396072316014  hr: 0  min: 40  sec: 54
epoch: 1 / 8  batch: 307 / 1224  loss: 0.8057534878145212  hr: 0  min: 40  sec: 49
epoch: 1 / 8  batch: 308 / 1224  loss: 0.8043592159043659  hr: 0  min: 40  sec: 45
epoch: 1 / 8  batch: 309 / 1224  loss: 0.8038092501533841  hr: 0  min: 40  sec: 42
epoch: 1 / 8  batch: 310 / 1224  loss: 0.8034417364866503  hr: 0  min: 40  sec: 43
epoch: 1 / 8  batch: 311 / 1224  loss: 0.8034508884719713  hr: 0  min: 40  sec: 39
epoch: 1 / 8  batch: 312 / 1224  loss: 0.8031820122821208  hr: 0  min: 40  sec: 38
epoch: 1 / 8  batch: 313 / 1224  loss: 0.8016619336681244  hr: 0  min: 40  sec: 40
epoc

epoch: 1 / 8  batch: 401 / 1224  loss: 0.766380537477812  hr: 0  min: 40  sec: 48
epoch: 1 / 8  batch: 402 / 1224  loss: 0.7657253770389367  hr: 0  min: 40  sec: 49
epoch: 1 / 8  batch: 403 / 1224  loss: 0.7658679143666629  hr: 0  min: 40  sec: 51
epoch: 1 / 8  batch: 404 / 1224  loss: 0.7669145790952268  hr: 0  min: 40  sec: 53
epoch: 1 / 8  batch: 405 / 1224  loss: 0.7669688951821975  hr: 0  min: 40  sec: 51
epoch: 1 / 8  batch: 406 / 1224  loss: 0.7673102692136624  hr: 0  min: 40  sec: 49
epoch: 1 / 8  batch: 407 / 1224  loss: 0.7670244556972963  hr: 0  min: 40  sec: 49
epoch: 1 / 8  batch: 408 / 1224  loss: 0.7671794026505714  hr: 0  min: 40  sec: 47
epoch: 1 / 8  batch: 409 / 1224  loss: 0.766701853654204  hr: 0  min: 40  sec: 50
epoch: 1 / 8  batch: 410 / 1224  loss: 0.7661920393385538  hr: 0  min: 40  sec: 52
epoch: 1 / 8  batch: 411 / 1224  loss: 0.7654841018709243  hr: 0  min: 40  sec: 53
epoch: 1 / 8  batch: 412 / 1224  loss: 0.7658530059948708  hr: 0  min: 40  sec: 55
epoch:

...

epoch: 8 / 8  batch: 746 / 1224  loss: 0.09382400790544042  hr: 0  min: 2  sec: 10
epoch: 8 / 8  batch: 747 / 1224  loss: 0.09370063014896998  hr: 0  min: 2  sec: 9
epoch: 8 / 8  batch: 748 / 1224  loss: 0.09359077662030324  hr: 0  min: 2  sec: 9
epoch: 8 / 8  batch: 749 / 1224  loss: 0.0940049910477707  hr: 0  min: 2  sec: 9
epoch: 8 / 8  batch: 750 / 1224  loss: 0.09453395333886147  hr: 0  min: 2  sec: 9
epoch: 8 / 8  batch: 751 / 1224  loss: 0.09441802388778857  hr: 0  min: 2  sec: 8
epoch: 8 / 8  batch: 752 / 1224  loss: 0.0943585369200941  hr: 0  min: 2  sec: 8
epoch: 8 / 8  batch: 753 / 1224  loss: 0.09500279699070045  hr: 0  min: 2  sec: 8
epoch: 8 / 8  batch: 754 / 1224  loss: 0.0948825739521721  hr: 0  min: 2  sec: 7
epoch: 8 / 8  batch: 755 / 1224  loss: 0.09498521027383425  hr: 0  min: 2  sec: 7
epoch: 8 / 8  batch: 756 / 1224  loss: 0.09489787694204736  hr: 0  min: 2  sec: 7
epoch: 8 / 8  batch: 757 / 1224  loss: 0.0948044954647791  hr: 0  min: 2  sec: 7
epoch: 8 / 8  batch

epoch: 8 / 8  batch: 846 / 1224  loss: 0.09574228156155447  hr: 0  min: 1  sec: 42
epoch: 8 / 8  batch: 847 / 1224  loss: 0.09563648124547607  hr: 0  min: 1  sec: 42
epoch: 8 / 8  batch: 848 / 1224  loss: 0.0957293004623421  hr: 0  min: 1  sec: 41
epoch: 8 / 8  batch: 849 / 1224  loss: 0.09565538900725552  hr: 0  min: 1  sec: 41
epoch: 8 / 8  batch: 850 / 1224  loss: 0.09574351523728931  hr: 0  min: 1  sec: 41
epoch: 8 / 8  batch: 851 / 1224  loss: 0.09599714213161435  hr: 0  min: 1  sec: 41
epoch: 8 / 8  batch: 852 / 1224  loss: 0.0961980222518875  hr: 0  min: 1  sec: 40
epoch: 8 / 8  batch: 853 / 1224  loss: 0.09620793075557331  hr: 0  min: 1  sec: 40
epoch: 8 / 8  batch: 854 / 1224  loss: 0.09613008056472838  hr: 0  min: 1  sec: 40
epoch: 8 / 8  batch: 855 / 1224  loss: 0.09606506487256603  hr: 0  min: 1  sec: 40
epoch: 8 / 8  batch: 856 / 1224  loss: 0.09632927893930784  hr: 0  min: 1  sec: 39
epoch: 8 / 8  batch: 857 / 1224  loss: 0.09718258584150773  hr: 0  min: 1  sec: 39
epoch:

epoch: 8 / 8  batch: 946 / 1224  loss: 0.09638299830809204  hr: 0  min: 1  sec: 14
epoch: 8 / 8  batch: 947 / 1224  loss: 0.09628595119104214  hr: 0  min: 1  sec: 14
epoch: 8 / 8  batch: 948 / 1224  loss: 0.0961895361699887  hr: 0  min: 1  sec: 14
epoch: 8 / 8  batch: 949 / 1224  loss: 0.09621788602169246  hr: 0  min: 1  sec: 14
epoch: 8 / 8  batch: 950 / 1224  loss: 0.09612503009407145  hr: 0  min: 1  sec: 13
epoch: 8 / 8  batch: 951 / 1224  loss: 0.09603239295046412  hr: 0  min: 1  sec: 13
epoch: 8 / 8  batch: 952 / 1224  loss: 0.09594238563558134  hr: 0  min: 1  sec: 13
epoch: 8 / 8  batch: 953 / 1224  loss: 0.09585272910900156  hr: 0  min: 1  sec: 13
epoch: 8 / 8  batch: 954 / 1224  loss: 0.09575847773744375  hr: 0  min: 1  sec: 12
epoch: 8 / 8  batch: 955 / 1224  loss: 0.09598752068287415  hr: 0  min: 1  sec: 12
epoch: 8 / 8  batch: 956 / 1224  loss: 0.0959816806828876  hr: 0  min: 1  sec: 12
epoch: 8 / 8  batch: 957 / 1224  loss: 0.09588776239220251  hr: 0  min: 1  sec: 11
epoch:

epoch: 8 / 8  batch: 1045 / 1224  loss: 0.09932676843906704  hr: 0  min: 0  sec: 48
epoch: 8 / 8  batch: 1046 / 1224  loss: 0.09925570200067406  hr: 0  min: 0  sec: 47
epoch: 8 / 8  batch: 1047 / 1224  loss: 0.09919047881533333  hr: 0  min: 0  sec: 47
epoch: 8 / 8  batch: 1048 / 1224  loss: 0.09910934082168432  hr: 0  min: 0  sec: 47
epoch: 8 / 8  batch: 1049 / 1224  loss: 0.09904038394003169  hr: 0  min: 0  sec: 47
epoch: 8 / 8  batch: 1050 / 1224  loss: 0.09898155921271869  hr: 0  min: 0  sec: 46
epoch: 8 / 8  batch: 1051 / 1224  loss: 0.0988990373477042  hr: 0  min: 0  sec: 46
epoch: 8 / 8  batch: 1052 / 1224  loss: 0.09883219027048962  hr: 0  min: 0  sec: 46
epoch: 8 / 8  batch: 1053 / 1224  loss: 0.09874109625957857  hr: 0  min: 0  sec: 46
epoch: 8 / 8  batch: 1054 / 1224  loss: 0.09892857188546228  hr: 0  min: 0  sec: 45
epoch: 8 / 8  batch: 1055 / 1224  loss: 0.09890609409713066  hr: 0  min: 0  sec: 45
epoch: 8 / 8  batch: 1056 / 1224  loss: 0.09883202858636099  hr: 0  min: 0  s

epoch: 8 / 8  batch: 1143 / 1224  loss: 0.09762227966116899  hr: 0  min: 0  sec: 21
epoch: 8 / 8  batch: 1144 / 1224  loss: 0.09755828237434576  hr: 0  min: 0  sec: 21
epoch: 8 / 8  batch: 1145 / 1224  loss: 0.09749709115928958  hr: 0  min: 0  sec: 21
epoch: 8 / 8  batch: 1146 / 1224  loss: 0.09748421493027015  hr: 0  min: 0  sec: 20
epoch: 8 / 8  batch: 1147 / 1224  loss: 0.09743392118679095  hr: 0  min: 0  sec: 20
epoch: 8 / 8  batch: 1148 / 1224  loss: 0.09742241942742352  hr: 0  min: 0  sec: 20
epoch: 8 / 8  batch: 1149 / 1224  loss: 0.0973537123143621  hr: 0  min: 0  sec: 20
epoch: 8 / 8  batch: 1150 / 1224  loss: 0.09728153102423834  hr: 0  min: 0  sec: 19
epoch: 8 / 8  batch: 1151 / 1224  loss: 0.09727662222396799  hr: 0  min: 0  sec: 19
epoch: 8 / 8  batch: 1152 / 1224  loss: 0.09721616265596822  hr: 0  min: 0  sec: 19
epoch: 8 / 8  batch: 1153 / 1224  loss: 0.09718755679164673  hr: 0  min: 0  sec: 18
epoch: 8 / 8  batch: 1154 / 1224  loss: 0.09716161525771151  hr: 0  min: 0  s

In [11]:
# Menampilkan nilai loss
history_df = pd.DataFrame(history)
history_df

,loss
0,0.600860
1,0.378505
2,0.233461
3,0.169255
4,0.142035
5,0.116222
6,0.102341
7,0.098994


In [12]:
model_ABSA = load_model(model_ABSA, model_name)

In [13]:
# Classification report
x, y = test_model_ABSA(test_loader)
print(classification_report(x, y, target_names=[str(i) for i in range(3)]))

              precision    recall  f1-score   support

           0       0.95      0.94      0.95       806
           1       0.90      0.87      0.88       808
           2       0.86      0.90      0.88       799

    accuracy                           0.90      2413
   macro avg       0.90      0.90      0.90      2413
weighted avg       0.90      0.90      0.90      2413

